<a href="https://colab.research.google.com/github/RokelisK123/1labor/blob/main/EN_LT_neural_machine_translation_with_keras_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# English-to-Lithuanian translation with KerasHub

**Author:** [Algirdas Laukaitis](https://github.com/aalgirdas/)<br>
**Date created:** 2024/05/26<br>
**Last modified:** 2024/10/30<br>
**Description:** Use KerasHub to train a sequence-to-sequence Transformer model on the machine translation task.

In [ ]:
#@title Imports

#from google.colab import files
#import pandas as pd
#import urllib.request




In [ ]:
import numpy as np
import random
import io
import requests
import zipfile
import os
import os.path
import re
from google.colab.data_table import DataTable
DataTable.max_columns = 50


## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

In [ ]:
!wget -N -O lit-eng.zip https://www.manythings.org/anki/lit-eng.zip

!unzip -o lit-eng.zip

text_file = "lit.txt"

for details.

--2024-11-07 12:48:06--  https://www.manythings.org/anki/lit-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83056 (81K) [application/zip]
Saving to: ‘lit-eng.zip’

lit-eng.zip         100%[===================>]  81.11K  --.-KB/s    in 0.03s   

2024-11-07 12:48:06 (2.46 MB/s) - ‘lit-eng.zip’ saved [83056/83056]

Archive:  lit-eng.zip
  inflating: _about.txt              
  inflating: lit.txt                 


In [ ]:
output_dir = '/content'
os.makedirs(output_dir, exist_ok=True)

zip_path = '/content/tempfile.zip'
response = requests.get('https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-lt.txt.zip')
#response = requests.get('https://object.pouta.csc.fi/OPUS-TildeMODEL/v2018/moses/en-lt.txt.zip')

with open(zip_path, 'wb') as file:
    file.write(response.content)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

#os.remove(zip_path)

print(f"Files extracted to {output_dir}")



Files extracted to /content


## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
Before adding the text to a list, we convert it to lowercase.

In [ ]:
alphabet_en = "abcdefghijklmnopqrstuvwxyz.,? '"
alphabet_lt = 'aąbcčdeęėfghiįyjklmnoprsštuųūvzž.,? '

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, lit, _ = line.split("\t")
    eng = eng.lower() ; eng = re.sub(r"[^" + alphabet_en + "]", " ", eng) ; eng = re.sub(r'\s+', ' ', eng)
    lit = lit.lower() ; lit = re.sub(r"[^" + alphabet_lt + "]", " ", lit) ; lit = re.sub(r'\s+', ' ', lit)
    text_pairs.append((eng, lit))

print(f'{len(text_pairs):>6}  translated examples. For example, example 111 {text_pairs[111]}')

  2211  translated examples. For example, example 111 ("i'm a loser.", 'aš nevykėlė.')


In [ ]:
# Some entries, such as 11111, come with bad LT encoding.
'''
with open('/content/OpenSubtitles.en-lt.en', 'r', encoding='utf-8') as file1, open('/content/OpenSubtitles.en-lt.lt', 'r', encoding='utf-8') as file2:
  for index,(line1, line2) in  enumerate(zip(file1, file2)):

    eng = line1.strip()
    lit = line2.strip()
    eng = eng.lower() ; eng = re.sub(r"[^" + alphabet_en + "]", " ", eng) ; eng = re.sub(r'\s+', ' ', eng)
    lit = lit.lower() ; lit = re.sub(r"[^" + alphabet_lt + "]", " ", lit) ; lit = re.sub(r'\s+', ' ', lit)

    text_pairs.append((eng, lit))

    #if index > 200000:
    #  break

print(f'{len(text_pairs):>6}  translated examples. For example, example 11111 {text_pairs[11111]}')

'''

'\nwith open(\'/content/OpenSubtitles.en-lt.en\', \'r\', encoding=\'utf-8\') as file1, open(\'/content/OpenSubtitles.en-lt.lt\', \'r\', encoding=\'utf-8\') as file2:\n  for index,(line1, line2) in  enumerate(zip(file1, file2)):\n\n    eng = line1.strip()\n    lit = line2.strip()\n    eng = eng.lower() ; eng = re.sub(r"[^" + alphabet_en + "]", " ", eng) ; eng = re.sub(r\'\\s+\', \' \', eng)\n    lit = lit.lower() ; lit = re.sub(r"[^" + alphabet_lt + "]", " ", lit) ; lit = re.sub(r\'\\s+\', \' \', lit)\n\n    text_pairs.append((eng, lit))\n\n    #if index > 200000:\n    #  break\n\nprint(f\'{len(text_pairs):>6}  translated examples. For example, example 11111 {text_pairs[11111]}\')\n\n'

Here's what our sentence pairs look like:

In [ ]:
# Some entries, such as 11111, come with bad LT encoding.
text_pairs[2211  -3 :2211  +3 ] + text_pairs[11111-3 :11111+3 ] + text_pairs[21111-3 :21111+3 ]


[("happiness depends, not so much on circumstances, as on one's way of looking at one's life.",
  'laimė priklauso ne tiek nuo išorinių aplinkybių, kiek nuo mūsų požiūrio į gyvenimą.'),
 ("before i get out of bed, i spend a little time thinking about what i'll be doing the rest of the day.",
  'prieš atsikeldamas iš lovos, aš praleidžiu kažkiek laiko galvodamas apie tai, ką veiksiu likusią dienos dalį.'),
 ('january, february, march, april, may, june, july, august, september, october, november and december are the twelve months of the year.',
  'metus sudaro dvylika mėnesių sausis, vasaris, kovas, balandis, gegužė, birželis, liepa, rugpjūtis, rugsėjis, spalis, lapkritis ir gruodis.')]

In [ ]:
for _ in range(5):
    print(random.choice(text_pairs))

('tom answered back.', 'tomas atsakė.')
('his pace quickened.', 'jo žingsniai pagreitėjo.')
('will you warm up the soup?', 'ar gali pašildyti sriubą?')
('their flag is blue and gold.', 'jų vėliava yra mėlynai auksinė.')
("what's your skype username?", 'koks tavo skaipo adresas?')


Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")


2211 total pairs
1549 training pairs
331 validation pairs
331 test pairs


## Setup

Before we start implementing the pipeline, let's import all the libraries we need.

In [ ]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-hub
!pip install -q --upgrade keras

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
##from tensorflow_text.tools.wordpiece_vocab import (  bert_vocab_from_dataset as bert_vocab,)


Let's also define our parameters/hyperparameters.

In [ ]:
BATCH_SIZE = 64
EPOCHS = 1  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40
ENG_VOCAB_SIZE = 15000
SPA_VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

## Tokenizing the data

We'll define two tokenizers - one for the source language (English), and the other
for the target language (Spanish). We'll be using
`keras_hub.tokenizers.WordPieceTokenizer` to tokenize the text.
`keras_hub.tokenizers.WordPieceTokenizer` takes a WordPiece vocabulary
and has functions for tokenizing the text, and detokenizing sequences of tokens.

Before we define the two tokenizers, we first need to train them on the dataset
we have. The WordPiece tokenization algorithm is a subword tokenization algorithm;
training it on a corpus gives us a vocabulary of subwords. A subword tokenizer
is a compromise between word tokenizers (word tokenizers need very large
vocabularies for good coverage of input words), and character tokenizers
(characters don't really encode meaning like words do). Luckily, KerasHub
makes it very simple to train WordPiece on a corpus with the
`keras_hub.tokenizers.compute_word_piece_vocabulary` utility.

In [ ]:

def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab


Every vocabulary has a few special, reserved tokens. We have four such tokens:

- `"[PAD]"` - Padding token. Padding tokens are appended to the input sequence
length when the input sequence length is shorter than the maximum sequence length.
- `"[UNK]"` - Unknown token.
- `"[START]"` - Token that marks the start of the input sequence.
- `"[END]"` - Token that marks the end of the input sequence.

In [ ]:

if os.path.exists('eng_vocab.txt'):
    with open('eng_vocab.txt', 'r') as file:
        eng_vocab = [line.strip() for line in file]
    with open('spa_vocab.txt', 'r') as file:
        spa_vocab = [line.strip() for line in file]

    eng_vocab = list(set(eng_vocab))
    spa_vocab = list(set(spa_vocab))
    print(f" vocab: { len(eng_vocab)}  { len(spa_vocab)} ")
else:
    print("Vocab does not exist.")


    reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

    eng_samples = [text_pair[0] for text_pair in train_pairs]
    eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

    spa_samples = [text_pair[1] for text_pair in train_pairs]
    spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)


    with open('eng_vocab.txt', 'w') as file:
        for item in eng_vocab:
            file.write(f"{item}\n")

    with open('spa_vocab.txt', 'w') as file:
        for item in spa_vocab:
            file.write(f"{item}\n")

Vocab does not exist.


Let's see some tokens!

In [ ]:
print("English Tokens: ", eng_vocab[0:1110])
print("Spanish Tokens: ", spa_vocab[0:1110])

English Tokens:  ['[PAD]', '[UNK]', '[START]', '[END]', "'", ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'the', 'you', 'to', '##s', 'tom', 'is', '##ing', 'in', '##y', 'he', 'it', '##d', 'that', '##er', 'do', 'can', 'this', 'have', '##e', '##ed', 'my', '##t', 'of', 'was', 'don', 'me', 'we', 'are', 'what', '##n', 'be', 'not', '##ly', 'for', 'she', '##en', 'want', '##k', 'like', 'your', 'on', '##r', 'did', 're', 'they', 'with', '##ce', '##on', 'how', 'there', '##p', 'know', 'very', 'will', '##m', '##ight', '##st', 'as', 'has', 'her', '##ry', 'him', 'his', 'll', '##ay', '##le', 'up', '##ion', '##ive', 'and', 'go', '##an', '##o', '##ow', '##one', 'about', 'all', 'at', 've', '##at', 'an', 'one', 'who', '##ake', '##al', '##ate', '##et', 'mary', '##g', '##ld', '##ss', 'than', 'where', '##ar', '##ould', 'by', 'never', 'see', '##ge', '##ong', '##ot', '##ut', '##ver', 'back', '##ch', '##ll', '##nt

Now, let's define the tokenizers. We will configure the tokenizers with the
the vocabularies trained above.

In [ ]:
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    ###vocabulary=eng_vocab, lowercase=False
    vocabulary=eng_vocab, lowercase=False, sequence_length=40,
)
spa_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=spa_vocab, lowercase=False, sequence_length=40,
)

Let's try and tokenize a sample from our dataset! To verify whether the text has
been tokenized correctly, we can also detokenize the list of tokens back to the
original text.

In [ ]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print( "Recovered text after detokenizing: ",  eng_tokenizer.detokenize(eng_tokens_ex),)

print()

spa_input_ex = text_pairs[0][1]
spa_tokens_ex = spa_tokenizer.tokenize(spa_input_ex)
print("LT sentence: ", spa_input_ex)
print("Tokens: ", spa_tokens_ex)
print(    "Recovered text after detokenizing: ",    spa_tokenizer.detokenize(spa_tokens_ex),)

English sentence:  i'm taking pictures of the bridge.
Tokens:  tf.Tensor(
[ 16   4  20  27 179 199  23 294 292  55 215  37  56  34   9  75 197 132
   6   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0], shape=(40,), dtype=int32)
Recovered text after detokenizing:  i ' m taking pictures of the bridge . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

LT sentence:  aš darau tilto nuotraukas.
Tokens:  tf.Tensor(
[ 39 163  63  25  42 209 116 264 108 184  47  97   5   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0], shape=(40,), dtype=int32)
Recovered text after detokenizing:  aš darau tilto nuotraukas . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


## Format datasets

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the tokenized source sentence and `decoder_inputs` is the target
sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target
sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

We will add special tokens, `"[START]"` and `"[END]"`, to the input Spanish
sentence after tokenizing the text. We will also pad the input to a fixed length.
This can be easily done using `keras_hub.layers.StartEndPacker`.

In [ ]:

def preprocess_batch(eng, spa):
    batch_size = ops.shape(spa)[0]

    eng = eng_tokenizer(eng)
    spa = spa_tokenizer(spa)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.

    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        ###pad_value=eng_tokenizer.token_to_id("[PAD]"),
        pad_value=0,
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `spa` and pad it as well.
    spa_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=spa_tokenizer.token_to_id("[START]"),
        end_value=spa_tokenizer.token_to_id("[END]"),
        ###pad_value=spa_tokenizer.token_to_id("[PAD]"),
        pad_value=0,
    )
    spa = spa_start_end_packer(spa)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 40 steps long):

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


In [ ]:
print(inputs.get( 'encoder_inputs')[0])
print(inputs.get( 'decoder_inputs')[0])
print(targets[0])

print(eng_tokenizer.detokenize(inputs.get( 'encoder_inputs')[0]))
print(spa_tokenizer.detokenize(inputs.get( 'decoder_inputs')[0]))
print(spa_tokenizer.detokenize(targets[0]))

tf.Tensor(
[115  56  54  11  75 242 268  39  36 115 218 131  34   8 297  75 154 179
   9 166 118 294  37   6   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0], shape=(40,), dtype=int32)
tf.Tensor(
[  2  27 207  58  24 300  76  71 131 237 139  22 241  76 148  35  85 132
  40  22  76 304 278  40 195   5   0   0   0   0   0   0   0   0   0   0
   0   0   0   0], shape=(40,), dtype=int32)
tf.Tensor(
[ 27 207  58  24 300  76  71 131 237 139  22 241  76 148  35  85 132  40
  22  76 304 278  40 195   5   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   3], shape=(40,), dtype=int32)
one of my dreams is to one day see the aurora borealis . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[START] viena mano svajonių kada nors pamatyti šiaurės pašvaistę . [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
viena mano svajonių kada nors pamatyti šiaurės pašvaistę . [PAD] [PAD] [PAD] [PAD] [PAD] [

## Building the model

Now, let's move on to the exciting part - defining our model!
We first need an embedding layer, i.e., a vector for every token in our input sequence.
This embedding layer can be initialised randomly. We also need a positional
embedding layer which encodes the word order in the sequence. The convention is
to add these two embeddings. KerasHub has a `keras_hub.layers.TokenAndPositionEmbedding `
layer which does all of the above steps for us.

Our sequence-to-sequence Transformer consists of a `keras_hub.layers.TransformerEncoder`
layer and a `keras_hub.layers.TransformerDecoder` layer chained together.

The source sequence will be passed to `keras_hub.layers.TransformerEncoder`, which
will produce a new representation of it. This new representation will then be passed
to the `keras_hub.layers.TransformerDecoder`, together with the target sequence
so far (target words 0 to N). The `keras_hub.layers.TransformerDecoder` will
then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking.
The `keras_hub.layers.TransformerDecoder` sees the entire sequence at once, and
thus we must make sure that it only uses information from target tokens 0 to N
when predicting token N+1 (otherwise, it could use information from the future,
which would result in a model that cannot be used at inference time). Causal masking
is enabled by default in `keras_hub.layers.TransformerDecoder`.

We also need to mask the padding tokens (`"[PAD]"`). For this, we can set the
`mask_zero` argument of the `keras_hub.layers.TokenAndPositionEmbedding` layer
to True. This will then be propagated to all subsequent layers.

In [ ]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)


##encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
###encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=SPA_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
###)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
)(decoder_sequence=x, encoder_sequence=encoder_outputs)

x = keras.layers.Dropout(0.5)(x)

decoder_outputs = keras.layers.Dense(SPA_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        ###encoded_seq_inputs,
        encoder_outputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'functional' (of type Functional) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [ ]:
if os.path.exists("/content/transformer_en_lt.keras"):
    transformer = keras.models.load_model("/content/transformer_en_lt.keras")
    print("The model exists.")
else:
    transformer.compile( "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    print("The model compiled.")

The model compiled.


## Training our model

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics,
rather than accuracy. However, in order to use metrics like ROUGE, BLEU, etc. we
will have decode the probabilities and generate the text. Text generation is
computationally expensive, and performing this during training is not recommended.

Here we only train for 1 epoch, but to get the model to actually converge
you should train for at least 10 epochs.

In [ ]:



transformer.summary()



Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, None, 256)      │      3,850,240 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, None, 15000)    │      9,283,992 │ decoder_inputs[1][0],  │
│                           │                        │                │ transformer_encoder[0… │
│                           │                        │                │ not_equal[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,449,304 (55.12 MB)

 Trainable params: 14,449,304 (55.12 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)
transformer.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['encoder_inputs', 'decoder_inputs']. Received: the structure of inputs={'encoder_inputs': '*', 'decoder_inputs': '*'}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'position_embedding' (of type PositionEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/la

25/25 ━━━━━━━━━━━━━━━━━━━━ 144s 5s/step - accuracy: 0.1738 - loss: 7.8064 - val_accuracy: 0.1939 - val_loss: 5.0916
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 134s 5s/step - accuracy: 0.3768 - loss: 5.0405 - val_accuracy: 0.4730 - val_loss: 4.7512
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 151s 5s/step - accuracy: 0.4037 - loss: 4.7466 - val_accuracy: 0.3956 - val_loss: 4.5898
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - accuracy: 0.4334 - loss: 4.5840 - val_accuracy: 0.3018 - val_loss: 4.5250
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 138s 5s/step - accuracy: 0.4533 - loss: 4.4555 - val_accuracy: 0.3816 - val_loss: 4.4336
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 130s 5s/step - accuracy: 0.4741 - loss: 4.3185 - val_accuracy: 0.2939 - val_loss: 4.3228
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 134s 5s/step - accuracy: 0.4713 - loss: 4.0458 - val_accuracy: 0.6131 - val_loss: 3.3089
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 150s 6s/step - accuracy: 0.5523 - loss: 2.8094 - val_accuracy: 0.6324 - val_loss: 1.

## Decoding test sentences (qualitative analysis)

Finally, let's demonstrate how to translate brand new English sentences.
We simply feed into the model the tokenized English sentence
as well as the target token `"[START]"`. The model outputs probabilities of the
next token. We then we repeatedly generated the next token conditioned on the
tokens generated so far, until we hit the token `"[END]"`.

For decoding, we will use the `keras_hub.samplers` module from
KerasHub. Greedy Decoding is a text decoding method which outputs the most
likely next token at each time step, i.e., the token with the highest probability.

In [ ]:

def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenize the encoder input.
    encoder_input_tokens = ops.convert_to_tensor(eng_tokenizer(input_sentences))
    #print(eng_tokenizer(input_sentences))
    #print(eng_tokenizer.detokenize(eng_tokenizer(input_sentences)))

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 40
    start = ops.full((batch_size, 1), spa_tokenizer.token_to_id("[START]"), dtype=int)
    #print(start)
    ###pad = ops.full((batch_size, length - 1), spa_tokenizer.token_to_id("[PAD]"))
    pad = ops.full((batch_size, length - 1), spa_tokenizer.token_to_id("[PAD]"), dtype=int)
    prompt = ops.concatenate((start, pad), axis=-1)
    print(prompt)

    generated_tokens = keras_hub.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[spa_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    generated_sentences = spa_tokenizer.detokenize(generated_tokens)
    return generated_sentences





In [ ]:
print(train_pairs[0][0])
print(train_pairs[0][1])
translated = decode_sequences([train_pairs[0][0]])
print(translated)

i'm taking pictures of the bridge.
aš darau tilto nuotraukas.
tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]], shape=(1, 40), dtype=int64)
['[START] , n vras negali rionme daugiaušte įč buvoiaurė ? [PAD]rė [PAD] , ? [PAD]rė [PAD]mekalė [PAD] v kiektą [PAD] p [PAD] ž [PAD]e']


In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]
for i in range(5):
    input_sentence = random.choice(test_eng_texts)
    print(input_sentence)
    translated = decode_sequences([input_sentence])
    ##translated = translated.numpy()[0].decode("utf-8")
    translated = translated[0]
    translated = (
        translated.replace("[PAD]", "") .replace("[START]", "") .replace("[END]", "") .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

come on. there's someone i want you to meet.
tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]], shape=(1, 40), dtype=int64)
** Example 0 **
come on. there's someone i want you to meet.
ęs ? labai , nėra buvontit jai šasekti ?            štu

it is love that rules the world.
tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]], shape=(1, 40), dtype=int64)
** Example 1 **
it is love that rules the world.
ašba tai tnkiteja įni noriai ? aš              . tiu     į

which one do you prefer?
tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]], shape=(1, 40), dtype=int64)
** Example 2 **
which one do you prefer?
d jogk š r buvoųlu a d      u       a k a   ų

too late.
tf.Tensor(
[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]], shape=(1, 40), dtype=int64)
** Example 3 **
too late.
mio ? ? ? ? ? mtą mkuku m      m   m      

In [ ]:
transformer.save("/content/transformer_en_lt.keras")